In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
import pprint
from bs4 import BeautifulSoup

In [2]:
# import apartment info from mongodb
db = client['apartments']
table = db['main_pages']

In [3]:
table.count_documents({})

586

In [4]:
apartment_collection = table.find()

In [11]:
#create new dataframe with just headers

# for each apartment in collection parse html and 
# find the following info in collection 
# [name, address, price or price range, and Walkscore] 
# and add to Pandas DF
for apartment in apartment_collection:
    soup = BeautifulSoup(apartment['html'], 'html.parser')
    
    listing_id = apartment['listing_id']
    property_name = soup.find('h1', 'propertyName').text.strip()
    address = soup.find('div', 'propertyAddress').find_all('span')[0].text
    print(address)
    break

110 Jacob Fontaine Lane


In [6]:
# Create column for cost/SF

# Create column for some metric comparing cost/SF and Walkscore. 
# Maybe it's the ((cost/SF) / Walkscore)